In [1]:
import caffe
import numpy as np

In [2]:
IROSpath = '/home/jguerry/workspace/IROSmodels/'

target_net_proto = '/home/jguerry/workspace/IROSmodels/fcn_vgg19bn_8s/deploy_37.prototxt'
target_weights = '/home/jguerry/workspace/IROSmodels/fcn_vgg19bn_8s/fcn_vgg19bn_8s.caffemodel'

source_net_proto = '/home/jguerry/workspace/IROSmodels/bn-models/VGG19_cvgj/deploy.prototxt'
source_weights = '/home/jguerry/workspace/IROSmodels/bn-models/VGG19_cvgj/vgg19_cvgj_iter_300000.caffemodel'

a_fcn_proto = '/home/jguerry/workspace/IROSmodels/fcn32s_HHA/deploy.prototxt'
a_fcn_weights = '/home/jguerry/workspace/IROSmodels/fcn32s_HHA/snapshot_iter_15900.caffemodel'


In [3]:
def transfer(new_net, net, suffix=''):
    # from fcn.berkeleyvision.org
    for p in net.params:
        #print "Working on", p, ':'
        p_new = p + suffix
        if p_new not in new_net.params:
            print 'dropping', p
            continue
        for i in range(len(net.params[p])):
            if i > (len(new_net.params[p_new]) - 1):
                print 'dropping', p, i
                break
            if net.params[p][i].data.shape != new_net.params[p_new][i].data.shape:
                print 'coercing', p, i, 'from', net.params[p][i].data.shape, 'to', new_net.params[p_new][i].data.shape
            else:
                print 'copying', p, ' -> ', p_new, i
            new_net.params[p_new][i].data.flat = net.params[p][i].data.flat

In [4]:
caffe_gpu = True
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [5]:
target_net = caffe.Net(target_net_proto, caffe.TEST)
target_net_blobs = [(k, v.data.shape) for k, v in target_net.blobs.items()]
target_net_layers = [(k, v[0].data.shape, v[1].data.shape) for k, v in target_net.params.items() if 'deconv' not in k]

In [6]:
source_net = caffe.Net(source_net_proto,source_weights, caffe.TEST)
source_net_blobs = [(k, v.data.shape) for k, v in source_net.blobs.items()]
source_net_layers = [(k, v[0].data.shape, v[1].data.shape) for k, v in source_net.params.items() if 'deconv' not in k]

In [7]:
transfer(target_net,source_net)

copying data/bn  ->  data/bn 0
copying data/bn  ->  data/bn 1
copying data/bn  ->  data/bn 2
copying data/scale  ->  data/scale 0
copying data/scale  ->  data/scale 1
copying conv1_1  ->  conv1_1 0
copying conv1_1  ->  conv1_1 1
copying conv1_1/bn  ->  conv1_1/bn 0
copying conv1_1/bn  ->  conv1_1/bn 1
copying conv1_1/bn  ->  conv1_1/bn 2
copying conv1_2  ->  conv1_2 0
copying conv1_2  ->  conv1_2 1
copying conv1_2/bn  ->  conv1_2/bn 0
copying conv1_2/bn  ->  conv1_2/bn 1
copying conv1_2/bn  ->  conv1_2/bn 2
copying conv2_1  ->  conv2_1 0
copying conv2_1  ->  conv2_1 1
copying conv2_1/bn  ->  conv2_1/bn 0
copying conv2_1/bn  ->  conv2_1/bn 1
copying conv2_1/bn  ->  conv2_1/bn 2
copying conv2_2  ->  conv2_2 0
copying conv2_2  ->  conv2_2 1
copying conv2_2/bn  ->  conv2_2/bn 0
copying conv2_2/bn  ->  conv2_2/bn 1
copying conv2_2/bn  ->  conv2_2/bn 2
copying conv3_1  ->  conv3_1 0
copying conv3_1  ->  conv3_1 1
copying conv3_1/bn  ->  conv3_1/bn 0
copying conv3_1/bn  ->  conv3_1/bn 1
copyi

In [8]:
#### SAVING
print "Saving output model to %s " % target_weights
target_net.save(target_weights)
print "Saved."

Saving output model to /home/jguerry/workspace/IROSmodels/fcn_vgg19bn_8s/fcn_vgg19bn_8s.caffemodel 


In [9]:
del target_net
del source_net